# Data Preprocessing - Data Download

Import packages

In [1]:
import json
import pandas as pd
from skimage import io
import h5py
import numpy as np
%cd ..

/Users/chen_bowen/Data Science/Udacity Courses/machine_learning_engineer/Capstone/outfit_completion_recommendation_system


Define data directory url and read json file to a dict

In [2]:
with open('data/external/fashion.json') as json_file:
    fashion_product_scene_mapping = [json.loads(line) for line in json_file]

with open('data/external/fashion-cat.json') as json_file:
    fashion_product_categories = [json.loads(line) for line in json_file]
    
with open('data/external/home.json') as json_file:
    home_product_scene_mapping = [json.loads(line) for line in json_file]

with open('data/external/home-cat.json') as json_file:
    home_product_categories = [json.loads(line) for line in json_file]

Use the `convert_to_url` function to get the url from the signature

In [3]:
def convert_to_url(signature):
    prefix = 'http://i.pinimg.com/400x/%s/%s/%s/%s.jpg'
    return prefix % (signature[0:2], signature[2:4], signature[4:6], signature)

save the data into two lists

In [4]:
from os import listdir
def image_retrieval(image_raw_mapping, image_cat= "fashion", image_type="product"):
    """ Retrieve all image data and save all """
    # remove duplicates
    image_raw_list = [i[image_type] for i in image_raw_mapping]
    image_raw_list_unique = list(dict.fromkeys(image_raw_list))
    output_img_path = "data/raw/{}_{}".format(image_cat, image_type)
    # get all unique images
    for signature in image_raw_list_unique:
        existed = [x[:-4] for x in listdir("data/raw/fashion_scene/")]
        if signature not in existed:
            image_url = convert_to_url(signature)
            try:
                img = io.imread(image_url)
                io.imsave(output_img_path + '/{}.jpg'.format(signature), img)
            except:
                print(image_url)

In [ ]:
image_retrieval(fashion_product_scene_mapping)
image_retrieval(fashion_product_scene_mapping, image_cat= "fashion", image_type="scene")
image_retrieval(home_product_scene_mapping, image_cat= "home", image_type="product")
image_retrieval(home_product_scene_mapping, image_cat= "home", image_type="scene")

http://i.pinimg.com/400x/ab/e0/fc/abe0fcbc61bde6dada7529677c80774e.jpg


get the max width and height for padding the images to the same shape

In [18]:
def max_width_height(image_list):
    h_w = np.array([[img.shape[0], img.shape[1]] for img in image_list])
    max_width = max(h_w[:, 1])
    max_height = max(h_w[:, 0])
    return max_height, max_width

In [19]:
max_width_height(image_list)

(882, 400)

pad all the images to the max height and width

In [23]:
fashion_product_scene_mapping[1000]

{'product': '588a44c6c2f5c79352fa01e84bec0a52',
 'scene': 'f86017bc2e570936f6850e92267e15f3',
 'bbox': [0.21987, 0.204131, 0.681583, 0.43806]}

Save all the images to a HDF5 file

In [8]:
def store_many_hdf5(images, metadata):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(f"{num_images}_products.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "meta", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    file.close()

In [9]:
store_many_hdf5(image_list, image_metadata_list)

OSError: Can't write data (no appropriate function for conversion path)